In [ ]:
import sys
import os

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    print("Detected Kaggle Environment")
    
    
    dataset_path = "/kaggle/input/helper-files" 
    
    if os.path.exists(dataset_path):
        sys.path.append(dataset_path)
        print(f"✅ Added to path: {dataset_path}")
        sys.path.append("/kaggle/input/helper-files/Dataset")
    else:
        print(f"❌ Error: Could not find dataset at {dataset_path}. Check the folder name!")

else:
    print("Detected Local Environment")

    current_dir = os.getcwd()

    parent_dir = os.path.dirname(current_dir)

    sys.path.append(parent_dir)

In [ ]:
import yfinance as yf
import pandas as pd
from Data.Slider import Slider
from Data import Data_prep
from models.LSTM import LSTM
from torch import nn
import torch
from tqdm import tqdm

In [ ]:

TICKER = 'Goldbees.ns'
START = '2009-01-02'
END = '2026-02-02'

data = yf.download(tickers = TICKER,
                   start = START,
                   end = END)
data

In [ ]:
df = pd.DataFrame(data)
df.set_index

In [ ]:
df.columns

In [ ]:
feat_cols = ['Open','High','Low','Volume']
feartures = df[feat_cols]
feartures = feartures.values

label_col = ['Close']
label = df[label_col]
label = label.values

In [ ]:
train_size = int(len(feartures)*0.8)
X_train = feartures[:train_size]
X_test = feartures[train_size:]
Y_train = label[:train_size]
Y_test = label[train_size:]
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

process_feat = MinMaxScaler(feature_range=(0,1))
process_targ = MinMaxScaler(feature_range=(0,1))

X_train = process_feat.fit_transform(X_train)
X_test = process_feat.fit_transform(X_test)

Y_train = process_targ.fit_transform(Y_train)
Y_test = process_targ.fit_transform(Y_test)

print(X_train[0],X_test[0],Y_train[0],Y_test[0])


In [ ]:
slidertr = Slider(feature = X_train,
                labels = Y_train,
                length = 60)

sliderts = Slider(feature = X_test,
                labels = Y_test,
                length = 60)

x_trainf,y_trainf = slidertr.slider()
x_testf,y_testf = sliderts.slider()

In [ ]:
#train data prep and load
x_t,y_t = Data_prep.convertNumpyToTensors(x_trainf,y_trainf)
train_dataset = Data_prep.createTensorDataset(x_t,y_t)
train_data_load = Data_prep.loadData(dataset=train_dataset,
                                               batch=32,
                                               num_worker=0)

In [ ]:
#test data pred and load
x_te,y_te = Data_prep.convertNumpyToTensors(x_testf,y_testf)
test_dataset = Data_prep.createTensorDataset(x_te,y_te)
test_data_load = Data_prep.loadData(dataset=test_dataset,
                                               batch=32,
                                               num_worker=0,
                                               shuffle=False)

In [ ]:
x,y = next(iter(train_data_load))
print(f'Shape of features {x.shape}')
print(f'Shape of target {y.shape}')

In [ ]:
device = 'cuda' if(torch.cuda.is_available()) else 'cpu'

In [ ]:
#model
INPUT_SIZE = 4
HIDDEN_UNITS = 64
OUT_FEATURES = 1
model = LSTM(in_size=INPUT_SIZE,
             hidden_units=HIDDEN_UNITS,
             out_features=OUT_FEATURES).to(device)

In [ ]:
#loss funtiona and Optimizer
loss_fn = nn.MSELoss()
optimizer =torch.optim.Adam(params=model.parameters(),
                            lr = 1e-4)

In [ ]:
#training and testing loop

def traintest(model:nn.Module,
          device:torch.device,
          Epoch:int,
          loss_fn:nn.MSELoss,
          optimizer:torch.optim.Adam,
          traindataloader:torch.utils.data.DataLoader,
          testdataloader: torch.utils.data.DataLoader,
          interval:int = 10):
    
  for epoch in range(Epoch):
    model.train()
    train_running_loss = 0.0
    test_running_loss = 0.0

    loop = iter(tqdm(traindataloader, desc = f'Epoch: {epoch+1}/{Epoch}'))

    for x,y in loop:
      x = x.to(device)  
      y = y.to(device)  
      train_pred = model(x)

      train_loss = loss_fn(train_pred,y) 
      train_running_loss += train_loss.item()

      optimizer.zero_grad()
      train_loss.backward()     
      optimizer.step()

    total_train_loss = train_running_loss/len(traindataloader)

    model.eval()
    with torch.inference_mode():
      
      for x,y in testdataloader:
        x = x.to(device)  
        y = y.to(device)  
        test_pred = model(x)

        test_loss = loss_fn(test_pred,y)
        test_running_loss += test_loss.item()

      total_test_loss = test_running_loss/len(testdataloader)

    if epoch%interval == 0:
      print(f'|Train Loss: {total_train_loss : 0.4f} | Test Loss: {total_test_loss: 0.4f} |')
    

In [ ]:
#training and testing
EPOCH=50
INTERVAL = 5
traintest(model=model,
          device=device,
          Epoch=EPOCH,
          loss_fn=loss_fn,
          optimizer=optimizer,
          traindataloader=train_data_load,
          testdataloader=test_data_load,
          interval=INTERVAL
          )